# DecisionCell  
A modular multi-agent framework for transparent, tool-augmented decision making The system combines:

- **ReAct-style reasoning** – agents think, take structured actions, call tools, and react to observations  
- **Transparent reasoning traces** – each agent exposes its step-by-step thought process  
- **Retrieval-Augmented Generation (RAG)** – agents can query their own knowledge bases to ground reasoning in data  

This notebook walks you through the full setup required to run a DecisionCell experiment.

In [1]:
from core.Agent import Agent
from core.Logbook import Logbook
from core.Tools import ask_human, agent_as_tool, kb_search, calculate
from core.Utils import save_logbook

# Load configs 
import yaml
from pathlib import Path

## 1. Set up the Staff (agents)

DecisionCell works with a small set of collaborating agents.  
Each agent is defined by:

- A **YAML config** (name, role, provider, model, system prompt)
- Optional **tools** attached at runtime (e.g. search, ask_human)
- A shared **logbook** used to record thoughts, actions, tool calls, and observations

In the next cell we create the global logbook,  load the configs, initialize the tools, and create the agents. 

In [2]:
# Create the staff logbook
logbook = Logbook()

# Create legal agent
legal_config = yaml.safe_load(Path("agents/config/LegalAdvisor.yaml").read_text(encoding="utf-8"))

search_legal = kb_search("agents/kb/LegalAdvisor", k=3)

legal = Agent(
    config = legal_config,
    logbook = logbook, 
    tools =  [ask_human, search_legal],
)

# Create cyber operations agent
cyber_operations_config = yaml.safe_load(Path("agents/config/CyberOperationsExpert.yaml").read_text(encoding="utf-8"))

search_cyber_operations = kb_search("agents/kb/CyberOperationsExpert", k=3)

cyber_operations = Agent(
    config = cyber_operations_config,
    logbook = logbook,
    tools =  [ask_human, search_cyber_operations],
)

# Create military operations agent
military_operations_config = yaml.safe_load(Path("agents/config/MilitaryOperationsExpert.yaml").read_text(encoding="utf-8"))

search_military_operations = kb_search("agents/kb/MilitaryOperationsExpert", k=3)

military_operations = Agent(
    config = military_operations_config,
    logbook = logbook,
    tools =  [ask_human, search_military_operations],
)

# Create commander agent
commander_config = yaml.safe_load(Path("agents/config/Commander.yaml").read_text(encoding="utf-8"))

ask_legal = agent_as_tool(legal, "ask_legal")
ask_cyber_operations = agent_as_tool(cyber_operations, "ask_cyber_operations")
ask_military_operations = agent_as_tool(military_operations, "ask_military_operations")

commander = Agent(
    config = commander_config,
    logbook = logbook,
    tools =  [ask_human, ask_legal, ask_cyber_operations, ask_military_operations],
)

c:\Users\marij\Documents\OpenUniversiteit\MasterAI\Generative AI\Opdracht 3 - Group Project\DecisionCell\core\Tools.py:34: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(persist_directory=str(kb_dir), embedding_function=embeddings)


In [3]:
# commander.reset()
# legal.reset()

## 2. Define the mission objective

The mission objective describes the operational problem to be analyzed. The objective is expressed as a multi-line natural-language description that
outlines the scenario and available Courses of Action (CoAs).  This text becomes  the first input to the Commander when the decision process begins.

In [4]:
objective = """ 
Select the single best Course of Action (CoA) to prevent an imminent attack by the Terrmisous group on a commercial cargo ship carrying chemical agents near the civilian port AricikPortus.

Choose ONE option only (CoA 1, CoA 2, or CoA 3). Do not combine options.

CoA 1: Disable the VicikPortus power grid using previously implanted BlackEnergy3-based malware to prevent loading.
CoA 2: Temporarily neutralize the VicikPortus civilian pump station using a DDoS attack exploiting an unpatched vulnerability, preventing fueling.
CoA 3: Conduct GNSS spoofing to alter the cargo ship’s position, velocity, and heading after departure so it cannot reach its intended target.

"""

## 3. Run the decision cycle

With the staff, tools, and mission objective defined, we can now execute the full decision cycle.  

In [5]:
commander.query(objective)

╭───────────────────────────────── | AGENT: Commander | ─────────────────────────────────╮
│  INPUT                                                                                 │
│                                                                                        │
│               Select the single best Course of Action (CoA) to prevent an imminent     │
│               attack by the Terrmisous group on a commercial cargo ship carrying       │
│               chemical agents near the civilian port AricikPortus.                     │
│                                                                                        │
│               Choose ONE option only (CoA 1, CoA 2, or CoA 3). Do not combine          │
│               options.                                                                 │
│                                                                                        │
│               CoA 1: Disable the VicikPortus power grid using previously implanted     │
│               BlackEnergy3-based malware to prevent loading.                           │
│               CoA 2: Temporarily neutralize the VicikPortus civilian pump station      │
│               using a DDoS attack exploiting an unpatched vulnerability, preventing    │
│               fueling.                                                                 │
│               CoA 3: Conduct GNSS spoofing to alter the cargo ship’s position,         │
│               velocity, and heading after departure so it cannot reach its intended    │
│               target.                                                                  │
│                                                                                        │
│                                                                                        │
│                                                                                        │
│  OUTPUT                                                                                │
│  Thought      I need to evaluate the three Courses of Action based on simplicity,      │
│               security, and proportionality to determine the best option to prevent    │
│               the imminent attack.                                                     │
│  Action       ask_cyber_operations                                                     │
│  Action line  Assess the feasibility and potential risks of CoA 1 (BlackEnergy3        │
│               malware), CoA 2 (DDoS attack on pump station), and CoA 3 (GNSS           │
│               spoofing) to disrupt the Terrmisous group's attack on the cargo ship.    │
│  Parse error  False                                                                    │
╰────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────── | AGENT: CyberOperationsExpert | ───────────────────────────╮
│  INPUT                                                                                 │
│               Assess the feasibility and potential risks of CoA 1 (BlackEnergy3        │
│               malware), CoA 2 (DDoS attack on pump station), and CoA 3 (GNSS           │
│               spoofing) to disrupt the Terrmisous group's attack on the cargo ship.    │
│                                                                                        │
│  OUTPUT                                                                                │
│  Thought      I need to evaluate the feasibility, potential effects, and risks of      │
│               each CoA option to disrupt the Terrmisous group's attack on the cargo    │
│               ship, considering the target system, access level, and potential         │
│               mitigations.                                                             │
│  Action       finish                                                                   │
│  Action line  COA OPTIONS: 1) BlackEnergy3 malware for disruption of pump control      │
│               systems, 2) DDoS attack on pump station for service denial, 3) GNSS      │
│               spoofing for navigation disruption | PROS: CoA-1 high impact on          │
│               industrial control, CoA-2 low technical barrier, CoA-3 potential for     │
│               widespread disruption | CONS: CoA-1 requires existing network access,    │
│               CoA-2 may not be targeted enough, CoA-3 complex to execute and           │
│               attribute | RECOMMENDATION: CoA-1 for high impact if access is           │
│               confirmed, otherwise CoA-2 for simplicity and speed, with careful        │
│               consideration of potential collateral damage and legal implications.     │
│  Parse error  False                                                                    │
╰────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── | AGENT: Commander | ─────────────────────────────────╮
│  INPUT                                                                                 │
│               Observation: finish: COA OPTIONS: 1) BlackEnergy3 malware for            │
│               disruption of pump control systems, 2) DDoS attack on pump station for   │
│               service denial, 3) GNSS spoofing for navigation disruption | PROS:       │
│               CoA-1 high impact on industrial control, CoA-2 low technical barrier,    │
│               CoA-3 potential for widespread disruption | CONS: CoA-1 requires         │
│               existing network access, CoA-2 may not be targeted enough, CoA-3         │
│               complex to execute and attribute | RECOMMENDATION: CoA-1 for high        │
│               impact if access is confirmed, otherwise CoA-2 for simplicity and        │
│               speed, with careful consideration of potential collateral damage and     │
│               legal implications.                                                      │
│                                                                                        │
│  OUTPUT                                                                                │
│  Thought      Based on the cyber operations expert's assessment, I must weigh the      │
│               pros and cons of each CoA, considering simplicity, security, and         │
│               proportionality, to select the best option to prevent the attack on the  │
│               cargo ship.                                                              │
│  Action       ask_legal                                                                │
│  Action line  Evaluate the legality of CoA 1 (BlackEnergy3 malware) and CoA 2 (DDoS    │
│               attack) under international law and the law of armed conflict, focusing  │
│               on potential collateral damage and proportionality.                      │
│  Parse error  False                                                                    │
╰────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────── | AGENT: LegalAdvisor | ────────────────────────────────╮
│  INPUT                                                                                 │
│               Evaluate the legality of CoA 1 (BlackEnergy3 malware) and CoA 2 (DDoS    │
│               attack) under international law and the law of armed conflict, focusing  │
│               on potential collateral damage and proportionality.                      │
│                                                                                        │
│  OUTPUT                                                                                │
│  Thought      I must assess the legality of CoA 1 and CoA 2 under international law,   │
│               considering potential collateral damage and proportionality principles.  │
│  Action       search                                                                   │
│  Action line  international law cyber warfare proportionality collateral damage        │
│               BlackEnergy3 DDoS                                                        │
│  Parse error  False                                                                    │
╰────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────── | AGENT: LegalAdvisor | ────────────────────────────────╮
│  INPUT                                                                                 │
│               Observation: Lexicon to                                                  │
│               AJP-01                                                                   │
│                Lex-5 Edition F Version 1                                               │
│               counterterrorism                                                         │
│               All preventive, defensive and offensive measures taken to reduce the     │
│               vulnerability of forces,                                                 │
│               individuals and property against terrorist threats and/or acts, and to   │
│               respond to terrorist                                                     │
│               acts.                                                                    │
│               Note: In the frame of the NATO Comprehensive Approach, these measures    │
│               can be                                                                   │
│               combined with or followed by measures enabling recovery after terrorist  │
│               acts.                                                                    │
│               (NATO Agreed)                                                            │
│                                                                                        │
│               cyberspace                                                               │
│               The global domain consisting of all interconnected communication,        │
│               information technology                                                   │
│               and other electronic systems, networks and their data, including those   │
│               which are separated                                                      │
│               or independent, which process, store or transmit data. (NATO Agreed)     │
│                                                                                        │
│               cyberspace operation                                                     │
│                                                                                        │
│               AJP-01                                                                   │
│                46 Edition F Version 1                                                  │
│               of high intensity contested engagements to neutralize or defend from     │
│               hostile action                                                           │
│               within cyberspace or space. Examples of major crisis response            │
│               operations are                                                           │
│               Operation Allied Force (Yugoslavia), Operation Deliberate Force          │
│               (Bosnia) and some                                                        │
│               offensive or defensive cyberspace operations.20                          │
│               2.54 Warfighting is not enduring; its role is to set the conditions for  │
│               other campaign                                                           │
│               themes and types of competition. In periods of strategic competition in  │
│               which adversaries                                                        │
│               merge the armed conflict and confrontation zones, warfighting rarely     │
│               ends with a cessation                                                    │
│               of violence and normal competition. Armed conflict’s destruction of      │
│               government and societal                                                  │
│               institutions can create conditions for intense rivalry and             

╭───────────────────────────────── | AGENT: Commander | ─────────────────────────────────╮
│  INPUT                                                                                 │
│               Observation: finish: COA OPTIONS: 1) BlackEnergy3 malware, 2) DDoS       │
│               attack | PROS: CoA-1 can disrupt enemy command and control, CoA-2 can    │
│               overwhelm enemy systems | CONS: CoA-1 may cause unintended harm to       │
│               civilian infrastructure, CoA-2 may violate principles of distinction     │
│               and proportionality | RECOMMENDATION: CoA-1 and CoA-2 must be carefully  │
│               planned and executed to minimize collateral damage and ensure            │
│               compliance with international humanitarian law, particularly Article 3   │
│               of the Hague Convention and the principles of distinction and            │
│               proportionality.                                                         │
│                                                                                        │
│  OUTPUT                                                                                │
│  Thought      Having considered the expert opinions on cyber operations and legal      │
│               implications, I can now decide on the best Course of Action to prevent   │
│               the imminent attack on the cargo ship, balancing simplicity, security,   │
│               and proportionality.                                                     │
│  Action       finish                                                                   │
│  Action line  DECISION: CoA-1 (BlackEnergy3 malware) | REJECTED: CoA-2 (DDoS attack),  │
│               CoA-3 (GNSS spoofing) | SIMPLICITY: Direct disruption of industrial      │
│               control systems | SECURITY: Lower risk of detection and attribution      │
│               compared to other options | PROPORTIONALITY: High impact on enemy        │
│               capabilities with careful planning to minimize collateral damage and     │
│               ensure compliance with international humanitarian law.                   │
│  Parse error  False                                                                    │
╰────────────────────────────────────────────────────────────────────────────────────────╯

'finish: DECISION: CoA-1 (BlackEnergy3 malware) | REJECTED: CoA-2 (DDoS attack), CoA-3 (GNSS spoofing) | SIMPLICITY: Direct disruption of industrial control systems | SECURITY: Lower risk of detection and attribution compared to other options | PROPORTIONALITY: High impact on enemy capabilities with careful planning to minimize collateral damage and ensure compliance with international humanitarian law.'

In [6]:
# Text
text = logbook.console.export_text(clear=False)

# HTML
html = logbook.console.export_html(inline_styles=True, clear=False)

# Raw trace
trace = logbook.trace

# Save results
save_logbook(text, html, trace)